In [1]:
import builtins
import copy
import time

from selenium import webdriver
from selenium.common import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
from selenium.common import StaleElementReferenceException

from itertools import combinations
from datetime import datetime

In [2]:
def print(*args, **kwargs):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    builtins.print(f"[{timestamp}] ", *args, **kwargs)

def get_timestamp():
    return str(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

In [3]:
EDGE_DRIVER_PATH = "edge_driver/msedgedriver.exe"
driver = webdriver.Edge(service=Service(EDGE_DRIVER_PATH))

In [4]:
driver.get("https://rustypot.com/coinflip")

In [5]:
def get_possible_bet_values(inventory_items_values, max_items=5):
    print(f"there are {len(inventory_items_values)} items in inventory with total value: {sum(inventory_items_values):.2f}$")
    possible_bet_values = {}
    for r in range(1, max_items + 1):
        for combo in combinations(inventory_items_values, r):
            s = sum(combo)
            if s in possible_bet_values:
                continue
            max_tax = s * 2 * 0.05
            good_combo = True
            for item_value in combo:
                if item_value < max_tax:
                    good_combo = False
            if good_combo:
                possible_bet_values[s] = list(combo)
    possible_bet_values_sorted = dict(sorted(possible_bet_values.items()))

    return possible_bet_values_sorted

In [6]:
def get_inventory_items():
    create_a_game_buttom = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "createCoinflipButton"))
    )
    create_a_game_buttom.click()

    try:
        inventory_items = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "InventoryItem"))
        )

        # inventory_items = inventory_items_div.find_elements(By.CLASS_NAME, "InventoryItem")
        inventory_items_values = []
        for inventory_item in inventory_items:
            inventory_items_values.append(float(inventory_item.text.split("$")[1]))

        driver.get("https://rustypot.com/coinflip")

        print(f"inventory items gathered: {inventory_items_values}", )

        return inventory_items_values
    except TimeoutException:
        print(f"Timed out waiting for inventory items")
        driver.get("https://rustypot.com/coinflip")
        return get_inventory_items()

In [12]:
inventory_items_values = get_inventory_items()

[2025-04-07 05:07:31]  inventory items gathered: [9.89, 1.19, 1.16, 1.11, 1.09, 1.08, 1.07, 1.06, 1.06, 1.06, 1.06, 1.06, 1.06, 1.05, 1.04, 1.04, 1.04, 1.04, 1.04, 1.04, 1.04, 1.04, 1.04, 1.03, 1.03, 1.03, 1.03, 1.03, 1.03, 1.03, 1.03, 1.03, 1.03, 1.02, 1.02, 1.02, 1.02, 1.02, 1.02, 1.02, 1.01, 1.01, 1.01, 1.01, 1.01, 1.01, 1.01, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.97, 0.97, 0.97, 0.96, 0.95, 0.94, 0.93, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.9, 0.9, 0.9, 0.9, 0.9, 0.89, 0.89, 0.87, 0.87, 0.87]


In [14]:
# inventory_items_values = [1.0, 2.0, 3.0, 4.0, 5.0] ### TEST WITH HARDCODED ITEMS
start = time.time()
possible_bet_values = get_possible_bet_values(inventory_items_values, max_items=5)
end = time.time()

print(f"{end - start:.2f} seconds")
possible_bet_values

[2025-04-07 05:07:55]  there are 103 items in inventory with total value: 111.58$
[2025-04-07 05:08:06]  11.63 seconds


{0.87: [0.87],
 0.89: [0.89],
 0.9: [0.9],
 0.92: [0.92],
 0.93: [0.93],
 0.94: [0.94],
 0.95: [0.95],
 0.96: [0.96],
 0.97: [0.97],
 0.98: [0.98],
 0.99: [0.99],
 1.0: [1.0],
 1.01: [1.01],
 1.02: [1.02],
 1.03: [1.03],
 1.04: [1.04],
 1.05: [1.05],
 1.06: [1.06],
 1.07: [1.07],
 1.08: [1.08],
 1.09: [1.09],
 1.11: [1.11],
 1.16: [1.16],
 1.19: [1.19],
 1.74: [0.87, 0.87],
 1.76: [0.89, 0.87],
 1.77: [0.9, 0.87],
 1.78: [0.89, 0.89],
 1.79: [0.92, 0.87],
 1.8: [0.93, 0.87],
 1.81: [0.94, 0.87],
 1.8199999999999998: [0.95, 0.87],
 1.82: [0.93, 0.89],
 1.83: [0.96, 0.87],
 1.8399999999999999: [0.97, 0.87],
 1.84: [0.92, 0.92],
 1.85: [0.98, 0.87],
 1.8599999999999999: [0.99, 0.87],
 1.87: [1.0, 0.87],
 1.88: [1.01, 0.87],
 1.89: [0.95, 0.94],
 1.8900000000000001: [1.02, 0.87],
 1.9: [1.03, 0.87],
 1.91: [0.97, 0.94],
 1.9100000000000001: [1.04, 0.87],
 1.92: [1.05, 0.87],
 1.93: [0.99, 0.94],
 1.9300000000000002: [1.06, 0.87],
 1.94: [1.07, 0.87],
 1.95: [1.01, 0.94],
 1.950000000000000

In [7]:
def get_bet_items(bet_values, coinflip_value, coinflip_items_values):
    for bet_value in bet_values:
        if (coinflip_value * 0.9 < bet_value <= coinflip_value * max_bet_percentage
                and min_bet_value < coinflip_value < max_bet_value
                # and len(coinflip_items_values) >= len(bet_values[bet_value]) # FORCE TO GET MORE ITEMS
            ):
            return bet_values[bet_value]

    return None


def join_coinflip(join_button):

    try:
        join_button.click()

        spinner_locator = (By.CLASS_NAME, "spinner")
        WebDriverWait(driver, 4).until(
            EC.invisibility_of_element_located(spinner_locator)
        )
    except TimeoutException:
        print(f"Inventory loading timed out during selecting bet items")
        driver.get("https://rustypot.com/coinflip")
        join_coinflip(join_button)


def select_bet_items(bet_items_values):
    inventory_items_locator = (By.CLASS_NAME, "InventoryItem")
    inventory_items = WebDriverWait(driver, 3).until(
        EC.presence_of_all_elements_located(inventory_items_locator)
    )

    for inventory_item in inventory_items:
        item_value = float(inventory_item.text.split("$")[1])
        if item_value in bet_items_values:
            inventory_item.click()
            bet_items_values.remove(item_value)
        if len(bet_items_values) == 0:
            break


def press_deposit_button():
    deposit_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//button[@onclick='requestCfDeposit()']"))
    )
    deposit_button.click()


def press_accept_deposit_button_site():
    accept_button = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//button[@onclick='closeAcceptDeposit()']"))
    )
    accept_button.click()

def press_accept_winnings_button_site():
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH,
                                "//button[@onclick='closeAcceptWinnings()']"))
    )
    accept_button.click()

def press_no_doubledown_button():
    cancel_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@id='DDCancel' and contains(@class, 'DDOption')]"))
    )
    cancel_button.click()


def accept_steam_trade(deposit, acceptWinning):

    # 1.open the new window
    WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)

    original_window = driver.current_window_handle
    new_window = [w for w in driver.window_handles if w != original_window][0]
    driver.switch_to.window(new_window)


    # 2.close if profile reported
    try:
        warning_element = WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((
                By.XPATH,
                "//*[contains(., 'have reported Blown for attempting trade scams')]"
            ))
        )
        ok_button = WebDriverWait(driver, 1).until(
            EC.element_to_be_clickable((
                By.XPATH,
                "//div[contains(@class, 'btn_grey_steamui') and span[text()='OK']]"
                ))
        )
        ok_button.click()
    except TimeoutException:
        pass

    # 3. Accept the trade
    confirm_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'content') and contains(text(), 'Confirm trade contents')]"))
    )
    confirm_button.click()
    time.sleep(0.25)



    if deposit:
        confirm_gift_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[span[text()='Yes, this is a gift']]"))
        )
        confirm_gift_button.click()
        time.sleep(0.25)
    elif acceptWinning:


        confirm_gift_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='btn_green_steamui btn_medium']"))
        )
        confirm_gift_button.click()
        time.sleep(0.25)


        try:
            trust_bot_element = WebDriverWait(driver, 1).until(
                EC.element_to_be_clickable((By.XPATH, "//div[span[contains(text(), 'Yes, I trust')]]"))
            )
            trust_bot_element.click()
            time.sleep(0.25)
        except TimeoutException:
            pass

    accept_trade_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "trade_confirmbtn"))
    )
    accept_trade_btn.click()
    time.sleep(2)

    driver.close()
    driver.switch_to.window(original_window)


def open_cf_window_and_wait_to_flip():
    view_button = coinflip_div.find_element(By.XPATH, ".//button[text()='View']")
    view_button.click()

    # Wait until the <p> inside #fliper-coin disappears
    try:
        p_element = driver.find_element(By.CSS_SELECTOR, "#fliper-coin > p")
        WebDriverWait(driver, 120).until(EC.staleness_of(p_element))
    except:
        # If <p> is already gone, just wait for the new state
        pass


    WebDriverWait(driver, 120).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#fliper-coin .flipper"))
    )


def get_winner():
    # Wait for the parent div to appear
    creator_div = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "creator-imgs")))

    # Wait for the child img.pick within that div
    color_img = WebDriverWait(driver, 10).until(lambda d: creator_div.find_element(By.XPATH, ".//img[@class='pick']"))

    # Get the src
    src = color_img.get_attribute("src")

     # Extract the color from the filename
    if "Red" in src:
        creator_color = "Red"
    elif "Black" in src:
        creator_color = "Black"
    else:  # creator_color = "Unknown"
        time.sleep(0.5)
        print(f"creator_color is {src}. Retry getting creator")
        return get_winner()

    # 1. Get the 'flipper' element
    flipper = driver.find_element(By.CLASS_NAME, "flipper")

    # 2. Get the 'transform' style attribute
    transform_style = flipper.get_attribute("style")  # e.g. "transform: rotateY(1620deg);"

    # 3. Extract the degree
    import re
    match = re.search(r'rotateY\((\d+)deg\)', transform_style)
    if match:
        degrees = int(match.group(1))
        # Normalize the rotation within 360°
        final_angle = degrees % 360

        # Every 180° flips the face, so even number of 180s = same side
        flips = (degrees // 180) % 2

        flip_face = "Red" if flips == 1 else "Black"
        print("Coin landed on:", flip_face)
    else:
        time.sleep(0.5)
        print("Couldn't determine coin face. Retry getting flipping face")
        return get_winner()

    return creator_color, flip_face

def get_coinflip_items_values(coinflip_items):
    img_elements = coinflip_items.find_elements(By.TAG_NAME, "img")
    coinflip_items_values = []
    for img in img_elements:
        title = img.get_attribute("data-original-title")
        if "|" in title:
            price_str = title.split("$")[-1]
            price = float(price_str)
            coinflip_items_values.append(price)

    return coinflip_items_values

def log_bet(bet_value, won_value, win, profit, inventory_value):
    f = open("logs.csv", "a")
    f.write(f"{get_timestamp()},"
            f"{bet_value:.2f},"
            f"{won_value:.2f},"
            f"{win},"
            f"{profit:.2f},"
            f"{inventory_value:.2f}\n")
    print(f"LOG APPEND: {get_timestamp()},"
            f"{bet_value:.2f},"
            f"{won_value:.2f},"
            f"{win},"
            f"{profit:.2f},"
            f"{inventory_value:.2f}")
    f.close()

In [10]:
max_bet_percentage = 1.1
tax_procentage = 0.05
min_bet_value = 4.0
# max_bet_value = 5 * (1 + (1 - max_bet_percentage))
max_bet_value = 10.1

In [12]:
inventory_items_values = get_inventory_items()
possible_bet_values = get_possible_bet_values(inventory_items_values)

while True:
    try:
        active_coinflips = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "ActiveCoinflips")))
        coinflip_row_divs = active_coinflips.find_elements(By.CLASS_NAME, "coinflip")

        for coinflip_div in coinflip_row_divs:
            try:
                coinflip_value = float(coinflip_div.get_attribute("coinflip-value"))
                coinflip_items = coinflip_div.find_element(By.CLASS_NAME, "coinflipGameItems")
            except StaleElementReferenceException:
                print(f"coinflip divs are stale. Retry getting coinflip rows")
                break

            # cf value + bet value ( max 95% of cf value)
            tax_value = (coinflip_value + coinflip_value * max_bet_percentage) * tax_procentage  # the taxed amount is the total amount of the coinflip

            if "items" in coinflip_items.text:
                # print("more than 5 items...")
                continue

            coinflip_items_values = get_coinflip_items_values(coinflip_items)

            if any(tax_value > item_value for item_value in coinflip_items_values):
                # print(f"not profitable")
                continue


            # to join a cf, i need to deposit minimum value
            # 1. get the bet items
            bet_items_values = get_bet_items(possible_bet_values, coinflip_value, coinflip_items_values)

            if bet_items_values is None:
                # no available bet for this cf based on our items
                continue

            ### Press join on available coinflip. if it is ongoing (only View button visible, skip)
            join_button = coinflip_div.find_element(By.TAG_NAME, "button")
            if "Join" not in join_button.text:
                continue

            print(f"JOINING {coinflip_value=} -> {coinflip_items_values=} -> min bet {coinflip_value * 0.9:.2f}$")

            join_coinflip(join_button)

            try:
                select_bet_items(copy.deepcopy(bet_items_values))
            except TimeoutException:
                print(f"Timed out when selecting bet items")
                driver.get("https://rustypot.com/coinflip")
                time.sleep(0.5)
                break

            press_deposit_button()

            press_accept_deposit_button_site()

            # try:
            #     # check if game already in progress
            #     element = driver.find_element(By.XPATH, '//div[@id="AcceptDepositBody"]/h4[text()="This game is already in progress!"]')
            #
            #     WebDriverWait(driver, 3).until(
            #         EC.presence_of_element_located(element)
            #     )
            #
            #     # if present, break and continue
            #     driver.get("https://rustypot.com/coinflip")
            #     time.sleep(0.5)
            #     break
            # except TimeoutException:
            #     pass


            accept_steam_trade(deposit=True, acceptWinning=False)


            print(f"JOINED bet_value = {sum(bet_items_values):.2f} -> {bet_items_values=}")
            # bets.append(coinflip_value)

            open_cf_window_and_wait_to_flip()

            creator_color, flip_face = get_winner()

            # extract flip color

            if creator_color == flip_face:
                print(f"flip was lost!")
                time.sleep(4) # wait to see the flip

                driver.get("https://rustypot.com/coinflip")
                time.sleep(1)
                for item in bet_items_values:
                    inventory_items_values.remove(item)

                log_bet(sum(bet_items_values), coinflip_value, 0, -sum(bet_items_values), sum(inventory_items_values))

                possible_bet_values = get_possible_bet_values(inventory_items_values)

            else:
                print(f"flip was won!")
                time.sleep(4)

                press_no_doubledown_button()

                press_accept_winnings_button_site()

                accept_steam_trade(deposit=False, acceptWinning=True)

                for item in coinflip_items_values:
                    inventory_items_values.append(item)

                log_bet(sum(bet_items_values), coinflip_value, 1, coinflip_value, sum(inventory_items_values))

                possible_bet_values = get_possible_bet_values(inventory_items_values)

            print(f"Back to searching")
    except Exception:
        print(f"stale element error -> maybe captch / cf already started / something || RESET")
        driver.get("https://rustypot.com/coinflip")

    time.sleep(1)

[2025-04-07 06:22:15]  inventory items gathered: [9.91, 9.54, 8.76, 5.16, 4.77, 4.65, 4.49, 3.55, 3.26, 3.26, 2.28, 2.26, 2.25, 2.13, 2.08, 1.86, 1.85, 1.8, 1.56, 1.44, 1.33, 1.31, 1.26, 1.09, 1.08, 1.07, 1.06, 1.05, 1.05, 1.04, 1.04, 1.04, 1.04, 1.04, 1.04, 1.03, 1.03, 1.03, 1.03, 1.03, 1.03, 1.03, 1.02, 1.02, 1.02, 1.02, 1.02, 1.02, 1.02, 1.02, 1.02, 1.01, 1.01, 1.01, 1.01, 1.01, 1.01, 1.0, 1.0, 1.0, 1.0, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.97, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.91, 0.86, 0.79, 0.79, 0.67]
[2025-04-07 06:22:15]  there are 97 items in inventory with total value: 154.06$
[2025-04-07 06:24:09]  JOINING coinflip_value=6.17 -> coinflip_items_values=[3.17, 3.0] -> min bet 5.55$
[2025-04-07 06:24:17]  JOINED bet_value = 5.56 -> bet_items_values=[4.77, 0.79]
[2025-04-07 06:24:37]  Coin landed on: Black
[2025-04-07 06:24:37]  flip was lost!


KeyboardInterrupt: 

In [ ]:
press_no_doubledown_button()
press_accept_winnings_button_site()
accept_steam_trade(deposit=False, acceptWinning=True)

In [ ]:
driver.switch_to.window(driver.window_handles[0])

In [29]:
driver.window_handles

['BD6BA315C86EBAD62E446E545E984F78', 'DCB5AC4B3BA125615A6C930D2390DCFF']

In [ ]:
confirm_button = WebDriverWait(driver, 10).until(
         EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'content') and contains(text(), 'Confirm trade contents')]"))
)
confirm_button.click()

In [28]:
log_bet(5.52,
        6.13,
        1,
        6.13,
       133.90 + 6.13 )

[2025-04-07 06:12:04]  LOG APPEND: 2025-04-07 06:12:04,5.52,6.13,1,6.13,140.03


In [13]:
log_bet(sum(bet_items_values), coinflip_value, 0, coinflip_value, sum(inventory_items_values))

[2025-04-07 06:24:49]  LOG APPEND: 2025-04-07 06:24:49,5.56,6.17,0,6.17,154.06


In [ ]:
trust_bot_element = WebDriverWait(driver, 1).until(
    EC.element_to_be_clickable((By.XPATH, "//div[span[contains(text(), 'Yes, I trust')]]"))
)
trust_bot_element.click()